## Check the test dynamic_offset.txt files for self-consistency

The expectation is that `calc_aca_from_targ(target, y_off, z_off, SI_ALIGN)` should be the same as the `aca` coordinate.  Here `y_off` is the sum of the two offsets `target_offset_y + aca_offset_y`.

Instead there is a residual which looks like the CHARACTERISTICS offset (around -3, +11 arcsec).

In [1]:
from __future__ import division, print_function
from astropy.table import Table
import numpy as np
from Quaternion import Quat
import chandra_aca
from chandra_aca import calc_aca_from_targ
import parse_cm

In [2]:
# SI_ALIGN from Matlab code
SI_ALIGN = chandra_aca.ODB_SI_ALIGN
SI_ALIGN

array([[  9.99999906e-01,  -3.37419984e-04,  -2.73439987e-04],
       [  3.37419984e-04,   9.99999943e-01,  -4.61320600e-08],
       [  2.73439987e-04,  -4.61320600e-08,   9.99999963e-01]])

In [3]:
def print_dq(q1, q2):
    """
    Print the difference between two quaternions
    """
    dq = q1.inv() * q2
    dr, dp, dy, _ = np.degrees(dq.q) * 2 * 3600
    print('droll={:6.2f}, dpitch={:6.2f}, dyaw={:6.2f} arcsec'.format(dr, dp, dy))

In [4]:
def check_obs(obs):
    """
    Check `obs` (which is a row out of the dynamic offsets table) for consistency
    between target and aca coordinates given the target and aca offsets and the
    SI_ALIGN alignment matrix
    """
    y_off = (obs['target_offset_y'] + obs['aca_offset_y']) / 3600
    z_off = (obs['target_offset_z'] + obs['aca_offset_z']) / 3600
    
    q_targ = Quat([obs['target_ra'], obs['target_dec'], obs['target_roll']])
    q_aca = Quat([obs['aca_ra'], obs['aca_dec'], obs['aca_roll']])
    
    q_aca_out = calc_aca_from_targ(q_targ, y_off, z_off, SI_ALIGN)
    print('Obsid={} detector={} '.format(obs['obsid'], obs['detector']), end='')
    print_dq(q_aca, q_aca_out)

In [5]:
dat = Table.read('MAY0916/MAY0916_sample_aimpoint_adjustment_file.txt', format='ascii')

In [6]:
dat[:3]

obsid,detector,chipx,chipy,chip_id,target_offset_y,target_offset_z,target_ra,target_dec,target_roll,aca_offset_y,aca_offset_z,aca_ra,aca_dec,aca_roll,mean_date,mean_t_ccd
int64,string48,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string168,float64
18225,ACIS-S,200.7,467.9,7,-90.0,-18.0,227.366667,7.555556,179.999539,27.6,13.13,227.372188,7.570852,179.999271,2016:131:11:37:27.816,-15.81
17146,ACIS-I,930.2,1009.6,3,-18.0,-18.0,299.698333,40.893111,114.998359,27.41,9.12,299.728928,40.876608,114.979974,2016:131:23:55:27.816,-16.41
18310,ACIS-I,970.0,980.0,3,-18.0,-18.0,265.21363,-28.29669,81.021791,10.81,-11.4,265.219367,-28.317856,81.024325,2016:132:04:17:51.816,-15.9


In [7]:
# Check each observation
for obs in dat:
    check_obs(obs)

Obsid=18225 detector=ACIS-S droll= -1.66, dpitch= -3.53, dyaw= 12.50 arcsec
Obsid=17146 detector=ACIS-I droll= -5.90, dpitch= -2.86, dyaw= 10.02 arcsec
Obsid=18310 detector=ACIS-I droll=  0.67, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18311 detector=ACIS-I droll=  0.68, dpitch= -2.85, dyaw= 10.01 arcsec
Obsid=18312 detector=ACIS-I droll=  0.71, dpitch= -2.86, dyaw= 10.02 arcsec
Obsid=18313 detector=ACIS-I droll=  0.69, dpitch= -2.86, dyaw= 10.02 arcsec
Obsid=18314 detector=ACIS-I droll=  0.70, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18315 detector=ACIS-I droll=  0.71, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18316 detector=ACIS-I droll=  0.73, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18317 detector=ACIS-I droll=  0.74, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18318 detector=ACIS-I droll=  0.72, dpitch= -2.86, dyaw= 10.02 arcsec
Obsid=18319 detector=ACIS-I droll=  0.74, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18320 detector=ACIS-I droll=  0.67, dpitch= -2.85, dyaw= 10.02 arcsec
Obsid=18108 

In [8]:
dat = Table.read('JUN2016/JUN2016B_dynamical_offsets.txt', format='ascii')

In [9]:
for obs in dat:
    check_obs(obs)

Obsid=17749 detector=ACIS-S droll= -6.55, dpitch= -3.33, dyaw= 11.96 arcsec
Obsid=18032 detector=ACIS-S droll=-45.87, dpitch= -3.35, dyaw= 11.94 arcsec
Obsid=18054 detector=ACIS-I droll= 15.65, dpitch= -2.64, dyaw=  9.47 arcsec
Obsid=17870 detector=ACIS-S droll=-12.47, dpitch= -3.33, dyaw= 11.95 arcsec
Obsid=17790 detector=ACIS-S droll= -8.30, dpitch= -3.33, dyaw= 11.96 arcsec
Obsid=18289 detector=ACIS-I droll= 17.38, dpitch= -2.65, dyaw=  9.48 arcsec
Obsid=17510 detector=ACIS-I droll= -8.38, dpitch= -2.65, dyaw=  9.46 arcsec


## Validate calc_aca_from_targ() using JUN0115 obsid 17058.

Validate the Ska function to transform from target coordinates to ACA pointing assuming an SI_ALIGN matrix and Y/Z offsets.

From the OR and maneuver summary file
```
OBS,
 ID=17058,TARGET=(215.015417,-49.595083,{PKS B1416-493}),
 DURATION=(64000.000000),PRIORITY=5,SI=ACIS-I,GRATING=NONE,SI_MODE=TE_00458,
 ROLL=(305.0),
 ACA_MODE=DEFAULT,TARGET_OFFSET=(-0.009167,0.001667),
 DITHER=(ON,0.002222,0.360000,0.000000,0.002222,0.509100,0.000000),
 SEGMENT=(1,58800.000000),PRECEDING=(P8504),MIN_ACQ=1,MIN_GUIDE=1
 
 
   FINAL ID:    1705800
                       FINAL ATTITUDE
      STOP TIME (GMT):   2015:154:22:57:25.930
             RA (deg):    214.98451436
            DEC (deg):    -49.59669395
           ROLL (deg):    304.97646792
        Dev. from
      Opt. Roll (deg):     -4.70028676
      Sun Angle (deg):    140.49525748
           Quaternion:      0.228750858213    0.511758087381   -0.709666300958    0.426790869340

```

In [10]:
q_aca = Quat([0.228750858213,    0.511758087381,   -0.709666300958,    0.426790869340])
q_targ = Quat([215.015417,-49.595083, q_aca.roll])  # approximation, but OK here
y_off, z_off = -0.009167,0.001667

In [11]:
q_aca_out = calc_aca_from_targ(q_targ, y_off, z_off, SI_ALIGN)

In [12]:
print_dq(q_aca, q_aca_out)

droll=-84.72, dpitch= -0.02, dyaw= -0.03 arcsec


#### Conclusion:  calc_aca_from_targ is working correctly